In [1]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.0

In [2]:
!pip install elasticsearch -q
!pip install tensorflow_text -q

     |████████████████████████████████| 225kB 2.8MB/s 
     |████████████████████████████████| 2.6MB 2.8MB/s 


In [3]:
from subprocess import Popen, PIPE, STDOUT
from elasticsearch import Elasticsearch
from elasticsearch import helpers

import os
import time
import pprint
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

In [4]:
es_server = Popen(
  ['elasticsearch-7.9.0/bin/elasticsearch'], 
  stdout = PIPE, stderr = STDOUT,
  preexec_fn = lambda: os.setuid(1))

In [6]:
!curl -X GET "localhost:9200/"

{
  "name" : "271446f74bc6",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "LFSDOsPKRlWTh6CfkfCdsw",
  "version" : {
    "number" : "7.9.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "a479a2a7fce0389512d6a9361301708b92dff667",
    "build_date" : "2020-08-11T21:36:48.204330Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [7]:
def gen_data():
  with open('/content/gdrive/My Drive/finch/es/free_chat/data/basic.txt') as f:
    for line in f:
      line = line.rstrip()
      q, a = line.split('<SEP>')
      yield {
        '_index': 'chatbot',
        'question': q,
        'answer': a,
        'question_embedding': encoder(q)[0].numpy(),}

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
es = Elasticsearch()
print(es.ping())

es.indices.create(index='chatbot')

mapping = {
  'properties': {
    'question': {
      'type': 'text',
    },
    'question_embedding': {
      'type': 'dense_vector',
      'dims': 512,
    },
  }
}
es.indices.put_mapping(body=mapping, index='chatbot')

encoder = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3')

helpers.bulk(es, gen_data())

True


(366, [])

In [ ]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  query_vector = encoder([text_inp])[0].numpy()
  script_query = {
  'script_score': {
    'query': {'match_all': {}},
    'script': {
      'source': "cosineSimilarity(params.query_vector, doc['question_embedding']) + 1.0",
      'params': {'query_vector': query_vector},
      }
    }
  }
  dsl = {
    'query': script_query,
    '_source': {'excludes': ['question_embedding']},
  }
  hits = es.search(index='chatbot', body=dsl)['hits']['hits']
  print('Match:', hits[0]['_source'])
  print('%.2f sec' % (time.time() - t0))
  print()

Input:hi


/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


Match: {'question': 'hello', 'answer': '你好呀'}
1.20 sec

Input:hello
Match: {'question': 'hello', 'answer': '你好呀'}
0.09 sec

Input:good morning
Match: {'question': '上午好', 'answer': '上午好'}
0.07 sec

Input:good evening
Match: {'question': '晚安', 'answer': '晚安'}
0.06 sec

Input:thank you
Match: {'question': '谢谢你', 'answer': '不用谢'}
0.07 sec

Input:thanks
Match: {'question': '谢谢你', 'answer': '不用谢'}
0.06 sec

Input:i like you
Match: {'question': '我爱你', 'answer': '我也爱你'}
0.06 sec

Input:i love you
Match: {'question': '我爱你', 'answer': '我也爱你'}
0.06 sec

Input:cold
Match: {'question': '冷', 'answer': '多穿衣服哈'}
0.06 sec

Input:i feel cold
Match: {'question': '冷冷的', 'answer': '多穿衣服哈'}
0.06 sec

Input:hot
Match: {'question': '好热', 'answer': '对啊, 热死人了'}
0.06 sec

Input:it is too hot
Match: {'question': '有点热', 'answer': '对啊, 热死人了'}
0.06 sec

Input:what is the weather
Match: {'question': '天气怎么样', 'answer': '天气还行 要查具体的吗'}
0.07 sec

Input:weather
Match: {'question': '天气', 'answer': '天气还行 要查具体的吗'}
0.06 sec

